In [21]:
import cv2                       #importing dependencies
import os
from google.colab import drive
drive.mount('/content/gdrive')   #mounting google drive

Mounted at /content/gdrive


**PROCESSING OUR DATA**


In [14]:
count=0
dir='/content/gdrive/My Drive/mask_data'
for items in os.listdir(dir):               #getting the total number of files in the training directory
  for item in os.listdir(os.path.join(dir,items)):
     count=count+1
  print("There are ",count," files in training",items)
  count=0

dir='/content/gdrive/My Drive/mask_testdata'
for items in os.listdir(dir):               #getting the total number of files in the test directory
  for item in os.listdir(os.path.join(dir,items)):
     count=count+1
  print("There are ",count," files in validation",items)
  count=0


There are  669  files in training without_mask
There are  678  files in training with_mask
There are  117  files in validation with_mask
There are  109  files in validation without_mask


In [3]:
categories=os.listdir(dir)
label_pics=[i for i in range(len(categories))]  #creating a label dictionary for future references
label=dict(zip(label_pics,categories))
label

{0: 'without_mask', 1: 'with_mask'}

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data='/content/gdrive/My Drive/mask_data'
test_data='/content/gdrive/My Drive/mask_testdata'
train_datagen=ImageDataGenerator(rescale=1.0/255,      #getting and augmenting our training data for better accuracy
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')                                  

test_datagen=ImageDataGenerator(rescale=1.0/255)

train_generator=train_datagen.flow_from_directory(train_data,    
                                                  batch_size=20,
                                                  class_mode='categorical',
                                                  target_size=(150,150))
test_generator=test_datagen.flow_from_directory(test_data,           #getting our testing data along with folder and subfolder names
                                                batch_size=20,
                                                class_mode='categorical',
                                                target_size=(150,150))

Found 1347 images belonging to 2 classes.
Found 226 images belonging to 2 classes.


In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


**MAKING OUR CONVOLUTIONAL NEURAL NETWORK**

In [0]:
model=Sequential([
        Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),  #Conv Layer 1
        MaxPooling2D(2,2),                                           #MaxPooling layer 1

        Conv2D(32,(3,3),activation='relu'),                          #Conv Layer 2         
        MaxPooling2D(2,2),                                           #MaxPooling Layer 2

        Conv2D(64,(3,3),activation='relu'),                          #Conv Layer 3
        MaxPooling2D(2,2),                                           #MaxPooling layer 3

        Conv2D(128,(3,3),activation='relu'),                         #Conv layer 4
        MaxPooling2D(2,2),                                           #MaxPooling layer 4  

        Flatten(),                                                   #First fully Connected Layer      
        Dropout(0.3),                                                #30% Dropout
        
        Dense(128,activation='relu'),                                #Second fully Connected Layer 
        Dense(2,activation='softmax')                                #2 Feature-Output layer      
])

In [0]:
#from tensorflow.keras.optimizers import RMSprop                      #used for tuning purposes only
model.compile(optimizer='adam',                                       #compiling model with these Hyperparameters
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
model.summary()                                                 #Summary of the CNN Model we created

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)      

**TRAINING OUR MODEL ON 30 EPOCHS**

In [18]:
history=model.fit(train_generator,validation_data=train_generator,  #Training starts
                  epochs=30,
                  validation_steps=50,
                  verbose=1
                  )

Epoch 1/30
68/68 [==============================] - 30s 444ms/step - loss: 0.0658 - accuracy: 0.9785 - val_loss: 0.0549 - val_accuracy: 0.9790
Epoch 2/30
68/68 [==============================] - 29s 429ms/step - loss: 0.0898 - accuracy: 0.9651 - val_loss: 0.2084 - val_accuracy: 0.9190
Epoch 3/30
68/68 [==============================] - 30s 439ms/step - loss: 0.0942 - accuracy: 0.9710 - val_loss: 0.0541 - val_accuracy: 0.9840
Epoch 4/30
68/68 [==============================] - 30s 435ms/step - loss: 0.0760 - accuracy: 0.9740 - val_loss: 0.0834 - val_accuracy: 0.9690
Epoch 5/30
68/68 [==============================] - 31s 453ms/step - loss: 0.0643 - accuracy: 0.9673 - val_loss: 0.0565 - val_accuracy: 0.9770
Epoch 6/30
68/68 [==============================] - 30s 437ms/step - loss: 0.0424 - accuracy: 0.9852 - val_loss: 0.0504 - val_accuracy: 0.9820
Epoch 7/30
68/68 [==============================] - 30s 435ms/step - loss: 0.0673 - accuracy: 0.9755 - val_loss: 0.0409 - val_accuracy: 0.9870

In [0]:
model.save('/content/gdrive/My Drive/mask_trained.h5')          #saving our model


**EVALUATING THE PERFORMNACE**

In [22]:
model=tf.keras.models.load_model('/content/drive/My Drive/mask_trained.h5')
eval=model.evaluate(train_generator)                            #evaluating our model based on the test data
print("Loss is : ",eval[0])
print("Accuracy is : ",eval[1])

OSError: ignored

**TESTING ON IMAGES**

In [12]:
import numpy as np                                           
face_img=cv2.imread('/content/gdrive/My Drive/Zuckerberg.jfif')  #testing on random images 
resized=cv2.resize(face_img,(150,150))                           #resizing as per our input size
normalized=resized/255.0
reshaped=np.reshape(normalized,(1,150,150,3))                    #reshaping the image
result=model.predict(reshaped)                                   #gives us 2 probabilities that sum to 1
labelled=np.argmax(result,axis=1)[0]                             #getting the max of the two probabilities
print(label[labelled])                                           #printing out from the the label dictionary

without_mask
